# Clustering of trajectories

In [ ]:
import shnitsel as sh
import shnitsel.xarray
import matplotlib.pyplot as plt
import xarray as xr

from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.drawOptions.addAtomIndices = True

In [ ]:
homologs = sh.open_frames('/tmp/homologs.nc')
homologs

In [ ]:
homologs.atXYZ.sel(cmpnd='A03').isel(frame=0).sh.to_mol(to2D=False)

Choose the four different possible dihedrals as descriptors. Concatenate them into a single DataArray.

In [ ]:
desc_list = [
        homologs.atXYZ.sh.dihedral(3, 1, 0, 5, deg=True),
        homologs.atXYZ.sh.dihedral(3, 1, 0, 2, deg=True),
        homologs.atXYZ.sh.dihedral(4, 1, 0, 5, deg=True),
        homologs.atXYZ.sh.dihedral(4, 1, 0, 2, deg=True),
]
for i, desc in enumerate(desc_list):
    desc_list[i] = desc.assign_coords(descriptor=desc.attrs['long_name'])
descriptors = xr.concat(desc_list, dim="descriptor")
descriptors

Calculate the PCA of these descriptors, reducing from 4 to 2 dimensions.  
We will also need one of the original dihedral vectors for labelling trajectories.

In [ ]:
dpca, dpca_obj = descriptors.sh.pca('descriptor', return_pca_object=True)
dpca['dih'] = desc_list[1]
dpca

The loadings, when plotted together with the PCA results, will allow us to understand the significance of different directions in reduced space.

In [ ]:
loadings = xr.DataArray(
    data=dpca_obj.components_,
    dims=['PC', 'descriptor'],
    coords=dict(
        descriptor=descriptors.descriptor
    )
)
loadings

In [ ]:
plt.rc('font', size=12)
plt.rc('lines', linewidth=2)

fig, axs = plt.subplot_mosaic([['I01', 'A01'],['A02', 'A03']], layout='constrained')

# Grey background:
for name, ax in axs.items():
    ax.set_title(name)
    for (cmpnd, trajid), traj in dpca.sh.mgroupby(['cmpnd', 'trajid']):
        ax.plot(traj[:,0], traj[:,1], c='#ccc')

# Trajectories as lines, coloured by end-dihedral for A02 & A03:
for (cmpnd, trajid), traj in dpca.sh.mgroupby(['cmpnd', 'trajid']):
    if cmpnd in {'I01', 'A01'}:
        c = '#7E5273'
    elif traj['dih'].isel(frame=-1) < 80:
        c = '#2c3e50'
    elif traj['dih'].isel(frame=-1) > 100:
        c = '#C4A000'
    axs[cmpnd].plot(traj[:,0], traj[:,1], c=c, lw=0.5)

# Loadings as arrows:
for label, pcs in loadings.groupby('descriptor'):
    pc1, pc2 = pcs.item(0), pcs.item(1)
    ax.annotate('', xy=(0, 0), xytext=(pc1, pc2), arrowprops=dict(arrowstyle="<-", shrinkA=0, shrinkB=0))
    ax.text(pc1*1.15, pc2*1.15, label)

fig.supxlabel('PC1')
fig.supylabel('PC2')

In [ ]:
plt.rc('font', size=16)
plt.rc('lines', linewidth=1)

fig, ax = plt.subplots(1, 1)
fig.set_size_inches(4, 5)

# Grey background:
for (cmpnd, trajid), traj in dpca.sh.mgroupby(['cmpnd', 'trajid']):
        ax.plot(traj[:,0], traj[:,1], c='#ccc', alpha=1)

# Trajectories as lines, coloured by end-dihedral:
for trajid, traj in dpca.sel(cmpnd='A03').groupby('trajid'):
    if traj['dih'].isel(frame=-1) < 80:
        c = '#2c3e50'
    elif traj['dih'].isel(frame=-1) > 100:
        c = '#C4A000'
    ax.plot(traj[:,0], traj[:,1], c=c)

# Loadings as arrows:
for label, pcs in loadings.groupby('descriptor'):
    pc1, pc2 = pcs.item(0), pcs.item(1)
    # Draw arrow:
    ax.annotate('', xy=(0, 0), xytext=(pc1, pc2), arrowprops=dict(arrowstyle="<-", shrinkA=0, shrinkB=0))
    ax.text(pc1*1.15, pc2*1.15, label)

ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
# fig.savefig('/nc/reports/2025-06-30_4-step_figure/3.pdf')